In [1]:
import os
import pydot
import graphviz
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-15 08:37:39.506997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 08:37:40.474085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-15 08:37:40.474190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [2]:
image_size = (128, 128)
batch_size = 16


train_ds = tf.keras.utils.image_dataset_from_directory(
    "/workspace/Shantan_Data/Bayesian2v/Auto1/Bayesian_data/Raw_data/2_parts",
    validation_split=None,
    subset=None,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/workspace/Shantan_Data/Bayesian2v/Auto1/Bayesian_data/Raw_data/1_parts",
    validation_split=None,
    subset=None,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
print(train_ds)
print(val_ds)


Found 30 files belonging to 2 classes.
Found 30 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


2023-03-15 08:37:41.907959: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 08:37:43.488906: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15726 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2023-03-15 08:37:43.490395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30971 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


In [3]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

epochs = 20

#callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),]
model.compile(
    optimizer=keras.optimizers.Adam(0.00005),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/20


2023-03-15 08:37:48.501719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8303
2023-03-15 08:37:49.461729: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55c3e5409320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-15 08:37:49.461776: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-03-15 08:37:49.461792: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-03-15 08:37:49.468084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-15 08:37:49.588572: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 [==============================] - 10s 990ms/step - loss: 0.6874 - accuracy: 0.4667 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 0s 265ms/step - loss: 0.6204 - accuracy: 0.7000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 0s 235ms/step - loss: 0.5879 - accuracy: 0.7333 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 0s 252ms/step - loss: 0.6224 - accuracy: 0.6333 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 5/20
2/2 [==============================] - 0s 231ms/step - loss: 0.4694 - accuracy: 0.8667 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/20
2/2 [==============================] - 0s 270ms/step - loss: 0.5408 - accuracy: 0.7667 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 7/20
2/2 [==============================] - 0s 279ms/step - loss: 0.4831 - accuracy: 0.8333 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 8/20
2/2 [==============